In [1]:
using CSV
using DataFrames
using Dates

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
forecast_date = "2020_12_21";

In [4]:
rawdata = DataFrame(CSV.File("../rawdata/forecast_hhs_$(forecast_date).csv"));

In [5]:
first(rawdata,3)

,hospital_name,collection_week,fips_code,hospitalization_percentage,hospitalization_2020-12-15,target_date
,String,String,Int64,Float64,Float64,String
1,U P M C WESTERN MARYLAND,12/4/20,24001,1.0,5.57596,12/15/20
2,UMD BALTIMORE WASHINGTON MEDICAL CENTER,12/4/20,24003,0.521651,10.3602,12/15/20
3,ANNE ARUNDEL MEDICAL CENTER,12/4/20,24003,0.478349,9.50021,12/15/20


In [6]:
abbrevs = ["MD", "UMD"]
function fixname(x)
    words = split(x, " ")
    words = [(w in abbrevs) ? w : titlecase(w) for w in words]
    x = join(words, " ")
    x = replace(x, "'S" => "'s")
    return x
end;

In [7]:
data = select(rawdata,
    :hospital_name => ByRow(fixname) => :hospital,
    :target_date => ByRow(x -> Date(x, dateformat"mm/dd/yy")+Year(2000)) => :date,
    "hospitalization_2020-12-15" => :admissions,
)
sort!(data, [:hospital, :date])
first(data, 5)

,hospital,date,admissions
,String,Date,Float64
1,Adventist Healthcare Fort Washington Medical Ctr,2020-12-15,1.37423
2,Adventist Healthcare Fort Washington Medical Ctr,2020-12-16,1.42605
3,Adventist Healthcare Fort Washington Medical Ctr,2020-12-17,1.42248
4,Adventist Healthcare Fort Washington Medical Ctr,2020-12-18,1.46596
5,Adventist Healthcare Fort Washington Medical Ctr,2020-12-19,1.48681


In [8]:
println(unique(data.hospital))

["Adventist Healthcare Fort Washington Medical Ctr", "Adventist Healthcare Shady Grove Medical Center", "Adventist Healthcare White Oak Medical Center", "Anne Arundel Medical Center", "Atlantic General Hospital", "Baltimore Convention Center Alternate Care Site", "Calverthealth Medical Center", "Carroll Hospital Center", "Doctors'  Community Hospital", "Frederick Health Hospital", "Garrett County Memorial Hospital", "Grace Medical Center, Inc", "Greater Baltimore Medical Center", "Holy Cross Germantown Hospital", "Holy Cross Hospital", "Howard County General Hospital", "Johns Hopkins Bayview Medical Center", "Johns Hopkins Hospital, The", "Medstar Franklin Square Medical Center", "Medstar Good Samaritan Hospital", "Medstar Harbor Hospital", "Medstar Montgomery Medical Center", "Medstar Saint Mary's Hospital", "Medstar Southern Maryland Hospital Center", "Medstar Union Memorial Hospital", "Mercy Medical Center Inc", "Meritus Medical Center", "Northwest Hospital Center", "Saint Agnes Hos

In [9]:
data |> CSV.write("../data/forecast_hhs_$(forecast_date).csv");